In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=d5094e68d472096f0bd195e4971c0850d48c19d1612fe01ef4d818aeaf6b65ce
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Soja_Sensores").getOrCreate()

In [3]:
df = spark.read.format('csv').option('header', True).load('sensores-iot.csv')

In [4]:
df.show()

+---+-----------+-----------+--------+--------------------+-----------+-----------+
|_c0|  device_id|temperature|humidity|           timestamp|   latitude|  longitude|
+---+-----------+-----------+--------+--------------------+-----------+-----------+
|  0|sensor-2231|         25|      49|2004-10-08 05:19:...| 39.0319385|-167.644666|
|  1|sensor-3869|         17|      42|2010-06-16 20:35:...| 43.4913955|  76.084671|
|  2|sensor-7079|         35|      58|2010-10-03 18:10:...|-39.9795415|-149.006886|
|  3|sensor-1163|         15|      64|2005-10-10 11:38:...| 28.1871005|-112.581419|
|  4|sensor-4483|         14|      65|2020-04-29 07:56:...| 89.2243515| -16.076939|
|  5|sensor-2821|         15|      52|2000-10-27 23:32:...|-55.3025525| -26.657210|
|  6|sensor-9805|         12|      37|2022-08-09 22:08:...| -25.912786| 122.693490|
|  7|sensor-1230|         10|      60|1970-02-02 17:32:...|-88.9300035|  33.377804|
|  8|sensor-4472|         11|      75|2014-02-22 06:42:...|-89.0426855| 120.

In [5]:
# from pyspark.sql.functions import col, to_date, year, month, dayofmonth

from pyspark.sql import functions as F

In [6]:
df = df.withColumn('date', F.to_date(F.col('timestamp')))
df.show(5)

+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+
|_c0|  device_id|temperature|humidity|           timestamp|   latitude|  longitude|      date|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+
|  0|sensor-2231|         25|      49|2004-10-08 05:19:...| 39.0319385|-167.644666|2004-10-08|
|  1|sensor-3869|         17|      42|2010-06-16 20:35:...| 43.4913955|  76.084671|2010-06-16|
|  2|sensor-7079|         35|      58|2010-10-03 18:10:...|-39.9795415|-149.006886|2010-10-03|
|  3|sensor-1163|         15|      64|2005-10-10 11:38:...| 28.1871005|-112.581419|2005-10-10|
|  4|sensor-4483|         14|      65|2020-04-29 07:56:...| 89.2243515| -16.076939|2020-04-29|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+
only showing top 5 rows



In [7]:
df = (df.withColumn('year', F.year(F.col('date')))
          .withColumn('month', F.month(F.col('date')))
          .withColumn('day', F.dayofmonth(F.col('date')))
)

df.show()

+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|_c0|  device_id|temperature|humidity|           timestamp|   latitude|  longitude|      date|year|month|day|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|  0|sensor-2231|         25|      49|2004-10-08 05:19:...| 39.0319385|-167.644666|2004-10-08|2004|   10|  8|
|  1|sensor-3869|         17|      42|2010-06-16 20:35:...| 43.4913955|  76.084671|2010-06-16|2010|    6| 16|
|  2|sensor-7079|         35|      58|2010-10-03 18:10:...|-39.9795415|-149.006886|2010-10-03|2010|   10|  3|
|  3|sensor-1163|         15|      64|2005-10-10 11:38:...| 28.1871005|-112.581419|2005-10-10|2005|   10| 10|
|  4|sensor-4483|         14|      65|2020-04-29 07:56:...| 89.2243515| -16.076939|2020-04-29|2020|    4| 29|
|  5|sensor-2821|         15|      52|2000-10-27 23:32:...|-55.3025525| -26.657210|2000-10-27|2000|   10| 27|
|  6|senso

In [8]:
mean_temp = df.select(F.mean(F.col("temperature"))).collect()[0][0]

In [9]:
mean_humd = df.select(F.mean(F.col("humidity"))).collect()[0][0]

In [10]:
print('mean temperature = ', mean_temp)
print('mean humidity = ', mean_humd)

mean temperature =  22.491527
mean humidity =  55.033291


In [11]:
df = df.fillna({'temperature': mean_temp, 'humidity': mean_humd})

In [12]:
df.count()

1000000

In [13]:
# se a temperatura estiver entre 0 e 40, use a temperatura, se não use a mean_temp.

df = df.withColumn('temperature', F.when(F.col('temperature').between(0, 40), F.col('temperature')).otherwise(F.lit(mean_temp)))

In [14]:
df.show()

+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|_c0|  device_id|temperature|humidity|           timestamp|   latitude|  longitude|      date|year|month|day|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|  0|sensor-2231|         25|      49|2004-10-08 05:19:...| 39.0319385|-167.644666|2004-10-08|2004|   10|  8|
|  1|sensor-3869|         17|      42|2010-06-16 20:35:...| 43.4913955|  76.084671|2010-06-16|2010|    6| 16|
|  2|sensor-7079|         35|      58|2010-10-03 18:10:...|-39.9795415|-149.006886|2010-10-03|2010|   10|  3|
|  3|sensor-1163|         15|      64|2005-10-10 11:38:...| 28.1871005|-112.581419|2005-10-10|2005|   10| 10|
|  4|sensor-4483|         14|      65|2020-04-29 07:56:...| 89.2243515| -16.076939|2020-04-29|2020|    4| 29|
|  5|sensor-2821|         15|      52|2000-10-27 23:32:...|-55.3025525| -26.657210|2000-10-27|2000|   10| 27|
|  6|senso

In [15]:
df.write.parquet('paquet_tradicional_sensores.parquet')

In [16]:
df.write.option('compression', 'snappy').parquet('paquet_snappy_sensores.parquet')

In [17]:
df.write.option('compression', 'gzip').parquet('paquet_gzip_sensores.parquet')

In [18]:
df.write.partitionBy('year', 'month').parquet('paquet_year_month_sensores.parquet')

In [19]:
spark.read.parquet('paquet_tradicional_sensores.parquet').show(5)
spark.read.parquet('paquet_snappy_sensores.parquet').show(5)
spark.read.parquet('paquet_gzip_sensores.parquet').show(5)
spark.read.parquet('paquet_year_month_sensores.parquet').show(5)

+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|_c0|  device_id|temperature|humidity|           timestamp|   latitude|  longitude|      date|year|month|day|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
|  0|sensor-2231|         25|      49|2004-10-08 05:19:...| 39.0319385|-167.644666|2004-10-08|2004|   10|  8|
|  1|sensor-3869|         17|      42|2010-06-16 20:35:...| 43.4913955|  76.084671|2010-06-16|2010|    6| 16|
|  2|sensor-7079|         35|      58|2010-10-03 18:10:...|-39.9795415|-149.006886|2010-10-03|2010|   10|  3|
|  3|sensor-1163|         15|      64|2005-10-10 11:38:...| 28.1871005|-112.581419|2005-10-10|2005|   10| 10|
|  4|sensor-4483|         14|      65|2020-04-29 07:56:...| 89.2243515| -16.076939|2020-04-29|2020|    4| 29|
+---+-----------+-----------+--------+--------------------+-----------+-----------+----------+----+-----+---+
only showi